In [14]:
import classes
import functies
import sympy as sp
import numpy as np

In [ ]:
#puntje 2: definiëren an de functies
k, omega, p_0, p_0_x, phi, x, t = sp.symbols('k omega p_0 p_0_x phi x t')

#max amplitude in functie van x 
drukamplitude_vergelijking = classes.vergelijking(p_0 * sp.sin(k*x + phi))
drukamplitude_parameters = ['k', 'p_0', 'phi']

def drukamplitude_model(X, parameters):
    K, P_0, PHI = parameters
    return P_0 * np.sin(K*X + PHI)

#amplitude in functie van tijd voor vaste x 
druktijd_vergelijking = classes.vergelijking(p_0*sp.cos(omega*t+phi)) #merk op dat dit niet per sé dezelfde phi is als voor x, maar is irrelevant voor de fitcode
druktijd_parameters = ['omega', 'p_0_x', 'phi']

def druktijd_model(T, parameters):
    omega, p_0, phi = parameters
    return p_0*np.cos(omega*T+phi)


In [ ]:
###sjoemeliere sjoemelare om de txt files met de data om te zetten in effectieve nuttige dataverwerkingen
#blah blah...

###TEST_DATA:

X = [0,10,20] #een vector met de x-coordinaten van de metingen
T = [ np.linspace(0,10), #een matrix met de tijden voor elke meting op elke x-coordinaat
      np.linspace(1,11),
      np.linspace(7, 27)
]

P = [ 0.5*np.sin(np.linspace(0,10)+3)+np.random.normal(0,1), #een matrix met coordinaten P[x_coordinaat][tijd] #random normale fouten ingevoegd voor de test
          np.sin(np.linspace(0,10)+np.random.normal(0,1)), 
      0.5*np.sin(np.linspace(0,10)-1)+np.random.normal(0,1)
]
P_fout = np.full(np.shape(P), 1) #een matrix van dezelfde vorm als P met de fouten in



xval = 0
def invals_amplitude(x,y):
    phiwaardes = [3,0,-1]
    return [1,1,phiwaardes[xval]] #op deze manier kunnen we handmatig de fasehoek vastleggen en zo de tekenfout bij P_0 voorkomen

def invals_vastex(x,y):
    return [0,0,0]

In [ ]:
###vind de drukamplitude voor een vaste x-coordinaat
def fit_vaste_x(X, T, P, P_fout):
    """
    @IN:
    matrices X, T P en P_fout als gedefinieerd in vorige opgave
    @OUT:
    een lijst met maximale amplitude in datapuntvorm, in de volgorde van de x-coordinaat waarin ze voorkomen, en de pulsatie in datapuntvorm
    """
    P_0_x = []
    omega_x = []
    for xval in range(len(X)):
        druk = P[xval]
        drukfout = P_fout[xval]
        tijd = T[xval]
        tijdfit = functies.fit(druktijd_parameters, druktijd_model, invals_amplitude, tijd, druk, drukfout)

        omega_x.append(functies.vector_to_datapunt(tijdfit[0].copy(), omega))
        omega_x = functies.mu_sigma(omega_x, omega)
        P_0_x.append(functies.vector_to_datapunt(tijdfit[1].copy(), p_0_x))

    return P_0_x, omega_x

#fix de drukamplitude naar de x-coordinaat
def fit_naar_x(X, P_0_x, P_0_x_ERR):
    fitwaardes = functies.fit(drukamplitude_parameters, drukamplitude_model, invals_vastex, X, P_0_x, P_0_x_ERR)
    K = functies.vector_to_datapunt(fitwaardes[0], k)
    P_0 = functies.vector_to_datapunt(fitwaardes[1], p_0)
    PHI = functies.vector_to_datapunt(fitwaardes[2], phi)
    return K, P_0, PHI

Raw output


TypeError: 'list' object is not callable